In [23]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
import psycopg2 as pg2
import psycopg2.extras as pgex
this_host='34.211.59.66'
this_user='postgres'
this_password='postgres'
conn = pg2.connect(host = this_host, 
                        user = this_user,
                        password = this_password)


sql_select = '''select cleaned_tweet 
                from tweets where (date_time >= NOW() - '1 day'::INTERVAL) ;'''
cur = conn.cursor(cursor_factory=pgex.RealDictCursor)
cur.execute(sql_select)
rows = cur.fetchall()
conn.close()
df = pd.DataFrame(rows)
df.reset_index(inplace = True)

In [24]:
import pickle
!pip install redis
import redis
redis_ip = '34.211.59.66'
r = redis.StrictRedis(redis_ip)
r.keys()

[b'tweet_SVD_fit',
 b'hashtags_countvec_fit_temp',
 b'hashtag_tfidf',
 b'temp_hash_count_12hr',
 b'hashtags_countvec_fit_transform',
 b'hashtags_tfidf_fit_transform',
 b'tweet_tfidf_fit',
 b'hashtags_countvec_fit',
 b'tweets_tfd_svd_pipe',
 b'tweet_tfidf_fit_transform']

In [25]:
tfd_svd_pipe = Pipeline([
    ('tfidf',TfidfVectorizer()),
    ('svd',TruncatedSVD(n_components=50))
])

In [26]:
tfd_svd_pipe.fit(df['cleaned_tweet'])

Pipeline(steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...TruncatedSVD(algorithm='randomized', n_components=50, n_iter=5,
       random_state=None, tol=0.0))])

In [27]:
tfd_svd_pipe = pickle.dumps(tfd_svd_pipe)
r.set('tweets_tfd_svd_pipe', tfd_svd_pipe)

True